In [39]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib_scalebar.scalebar import ScaleBar

import scipy
import tifffile

import pandas as pd

In [ ]:
#Import all data, set genes
fish_datasets = ['../SL027A_000h_output-XETG00045__0021448__4__20240131__210104/transcripts.parquet',
                '../SL027A_072h_hyppla_output-XETG00045__0021360__28__20240131__210109/transcripts.parquet',
                '../SL027A_144h_hyppla_output-XETG00045__0021360__40__20240131__210109/transcripts.parquet']

nuclei_images = ['../SL027A_000h_output-XETG00045__0021448__4__20240131__210104/morphology_mip.ome.tif',
                '../SL027A_072h_hyppla_output-XETG00045__0021360__28__20240131__210109/morphology_mip.ome.tif',
                '../SL027A_144h_hyppla_output-XETG00045__0021360__40__20240131__210109/morphology_mip.ome.tif']

genes = ['CD44', 'SOX2', 'VEGFA']
colors = [(250/255, 242/255, 133/255), (70/255, 130/255, 193/255), (197/255, 30/255, 84/255)]
x_limits = [(350,1250), (300,1500), (250,1250)]
y_limits = [(400,1200), (500,1350), (300,1200)]
sizes = [0.5, 1.5, 0.5]

In [ ]:
for i in range(len(fish_datasets)):
    
    #Filter and adjust the data according to the instruction in https://kb.10xgenomics.com/hc/en-us/articles/11636252598925-What-are-the-Xenium-image-scale-factors
    dots = pd.read_parquet(fish_datasets[i])
    dots = dots[(dots['qv'] >= 20) & (dots['cell_id'] != 'UNASSIGNED')]
              
    with tifffile.TiffFile(nuclei_images[i]) as tif:
        image = tif.series[0].levels[0].asarray()
    nuclei = scipy.ndimage.zoom(image, 0.2125)
    
    for g in range(len(genes)):
        
        #Plot
        plt.rcParams["figure.figsize"] = (10,10)
        plt.imshow(nuclei, cmap='Greys_r', alpha=1)
        
        plt.scatter(
            x = 'x_location', y = 'y_location',
            data = dots[dots['feature_name']==genes[g]],
            color = colors[g],
            s = sizes[g], alpha = 1)

        scalebar = ScaleBar(1, 'µm', fixed_value=200, color = 'white', location='lower right', font_properties={"size": 1})
        plt.gca().add_artist(scalebar)

        plt.axis('off')
        plt.xlim(x_limits[i])
        plt.ylim(y_limits[i])
        name = fish_datasets[i][fish_datasets[i].find('SL'):fish_datasets[i].find('output')] + genes[g] + '.png'
        plt.savefig('../Figure2/' + name, dpi=600, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close()